In [27]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq


In [57]:
path = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/%s/Results.txt'
path_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'
path_res_seq = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.fasta'
experiments = ['mrcv_mites','mrcv_all','sun_all','sun_mites']
path_genome = '/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa'
path_blast_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/blast_mites.csv'
path_mirbase = '/home/juan/Desktop/juan/bio/mirna_mite/data/mature.fa'

In [97]:
#Load shortstack results
dfs = {}
for experiment in experiments:
    experiment_path = path % (experiment,)
    df = pd.read_csv(experiment_path, sep='\t')
    df['Name'] = experiment + '_' + df['Name']
    print('Experiment:', experiment)
    print('All results:',len(df.index))
    df = df[df.MIRNA == 'Y']
    print('Only miRNAs:',len(df.index))
    df = df[df.Reads > 20]
    print('Only miRNAs +20 reads:',len(df.index))
    print('*' * 10)
    dfs[experiment] = df

Experiment: mrcv_mites
All results: 1211339
Only miRNAs: 39
Only miRNAs +20 reads: 22
**********
Experiment: mrcv_all
All results: 38532
Only miRNAs: 135
Only miRNAs +20 reads: 135
**********
Experiment: sun_all
All results: 157492
Only miRNAs: 154
Only miRNAs +20 reads: 154
**********
Experiment: sun_mites
All results: 1211339
Only miRNAs: 24
Only miRNAs +20 reads: 12
**********


In [98]:
dfs = [ v for v in dfs.values() ]

In [99]:
df = pd.concat(dfs)

In [100]:
df.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
125231,1D:147896240-147896319,mrcv_mites_MITE_T_120271|chr2D|74975911|749759...,80,45,0.353,1,1.000,+,UAUAUUUUGGUACGGAGGGAU,22,...,21,Y,-1.0,0,4,2,38,1,0,0
171085,2A:168984136-168984233,mrcv_mites_MITE_T_102616|chr1B|301273397|30127...,98,198,1.554,2,0.045,-,UCGGAAUUAGUUGACACUCAAA,86,...,22,Y,-1.0,4,1,1,39,89,63,1


In [101]:
df['seqname'] = df['#Locus'].str.split(':').str[0]
df['position'] = df['#Locus'].str.split(':').str[1]
df['start'] = df.position.str.split('-').str[0].astype(int)
df['end'] = df.position.str.split('-').str[1].astype(int)
df['duplicated'] = 0
df['seqlen'] = df.end - df.start
df['overlap_len'] = df.seqlen  * 0.1
df['overlap_len'] = df.overlap_len.astype(int)
df['start_overlap'] = df.start + df.overlap_len
df['end_overlap'] = df.end - df.overlap_len

In [102]:
print(len(df.index))

323


In [34]:
df[df.index.isin(['876749','876750','876751','27433'])]

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,24,seqname,position,start,end,duplicated,seqlen,overlap_len,start_overlap,end_overlap
876749,6A:23496429-23496595,MITE_T_64052|chr7B|176002823|176003000|AT|187|...,167,111,0.871,18,0.045,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496429-23496595,23496429,23496595,0,166,16,23496445,23496579
876750,6A:23496449-23496574,MITE_T_70309|chr6D|429354379|429354545|TA|23|F...,126,108,0.848,18,0.028,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496449-23496574,23496449,23496574,0,125,12,23496461,23496562
876751,6A:23496458-23496565,MITE_T_95046|chr2B|521974897|521975013|TATA|12...,108,107,0.840,18,0.019,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496458-23496565,23496458,23496565,0,107,10,23496468,23496555
27433,6A:23496419-23496608,Cluster_27434,190,111,0.871,18,0.045,-,UUAGAGGUUUCAAUACGGACU,82,...,4,6A,23496419-23496608,23496419,23496608,0,189,18,23496437,23496590


In [103]:
duplicated = []
for k,v in df.iterrows():
    if k in duplicated:
        continue
    other = df[df.index != k]
    other = other[other.seqname == v.seqname]
    other = other[(other.end_overlap >= v.start) & (other.start_overlap <= v.end)]
    if len(other.index) > 0:
        duplicated += other.index.tolist()

In [104]:
duplicated = set(duplicated)

In [105]:
df = df[~df.index.isin(duplicated)]
count_total = len(df.index)
count_total

253

In [107]:
df_mites = df[df.Name.str.contains('MITE')]
count_mites = len(df_mites.index)
count_mites

28

In [108]:
df_nomites = df[~df.Name.str.contains('MITE')]
count_nomites = len(df_nomites.index)
count_nomites

225

In [109]:
count_mites * 100 / count_total

11.067193675889328

In [110]:
#get sequences
df.sort_values(['seqname', 'start'], inplace=True)
df.to_csv(path_res, sep='\t', index=None)

In [111]:
fasta_seq = SeqIO.parse(path_genome, 'fasta')
   

In [ ]:
buffer_seqs = []
for record in fasta_seq:
    dff_extract = df[df.seqname == record.id]
    print(record.id, len(dff_extract.index))
    clean_seq = ''.join(str(record.seq).splitlines())
    for k,v in dff_extract.iterrows():
        start = min(v.start,v.end)
        end = max(v.start,v.end)
        new_seq = clean_seq[start:end]
        id = v.Name
        desc = record.id + "_" + str(start) + '_' + str(end)
        seq = SeqRecord(Seq(new_seq), id=id, description=desc)
        buffer_seqs.append(seq)
SeqIO.write(buffer_seqs, path_res_seq, "fasta")

1A 3


In [89]:
df_trep = pd.read_csv(path_blast_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs','score']
df_trep.columns = cols
df_trep = df_trep[df_trep.pident >= 80]
df_trep = df_trep[df_trep.qcovs >= 80]
print(len(df_trep.index))
df_trep.head(2)

682


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
0,Cluster_8835,MITE_1712,1,79,2,80,9,0,88.608,7.860000e-24,79,80,81,99,112
1,Cluster_8835,MITE_233,3,77,4,78,8,4,84.416,1.330000e-14,77,80,80,94,78


In [90]:
df_trep = df_trep[['qseqid','sseqid']]

In [91]:
df_trep['MITE'] = df_trep.groupby(['qseqid'])['sseqid'].transform(lambda x: ','.join(x))
df_trep = df_trep[['qseqid','sseqid','MITE']].drop_duplicates()
df_trep = df_trep[['qseqid','MITE']]

In [92]:
df_trep.head(2)

,qseqid,MITE
0,Cluster_8835,"MITE_1712,MITE_233,MITE_1717,MITE_1157"
1,Cluster_8835,"MITE_1712,MITE_233,MITE_1717,MITE_1157"


In [79]:
df_ = pd.merge(df, df_trep, how='left', left_on='Name', right_on='qseqid')

In [82]:
df_.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,position,start,end,duplicated,seqlen,overlap_len,start_overlap,end_overlap,qseqid,MITE
0,1A:309399489-309399664,Cluster_872,176,553,4.340,1,0.002,-,ACGGCAUAGAGGCACUGCAAA,327,...,309399489-309399664,309399489,309399664,0,175,17,309399506,309399647,NaN,NaN
1,1A:419025472-419025570,Cluster_1015,99,384,3.014,20,1.000,+,CGAAUGUAUUUUUUAUGGCUUG,249,...,419025472-419025570,419025472,419025570,0,98,9,419025481,419025561,NaN,NaN


In [ ]:
df_mirbase = pd.read_csv(path_mirbase_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
df_mirbase.columns = cols
print(len(df_mirbase.index))
df_mirbase.head(2)